In [1]:
import pandas as pd
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import pickle
from nltk.tokenize import word_tokenize
from sklearn.decomposition import NMF, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc, fbeta_score
from sklearn.model_selection import learning_curve, KFold, GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import normalize
import re
from unidecode import unidecode

import matplotlib.style as style
%matplotlib inline
plt.style.use('ggplot')

/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
with open('data/cleaned_df.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [3]:
df.head()

,song_owner,song_title,song_element,verse,artist
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6304 entries, 0 to 6303
Data columns (total 5 columns):
song_owner      6304 non-null object
song_title      6304 non-null object
song_element    6304 non-null object
verse           6304 non-null object
artist          6304 non-null object
dtypes: object(5)
memory usage: 246.3+ KB


In [5]:
model_df = df.copy()

model_df = model_df[model_df['artist'] == 'Drake']

In [6]:
model_df['verse_words'] = model_df['verse'].copy()
model_df['verse_words'] = model_df['verse_words'].str.lower()
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.replace(',',''))
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.split(' '))

In [7]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words
1282,Drake,Buried Alive (Interlude),[Intro],"If you was in a pine box, I would surely break...",Drake,"[if, you, was, in, a, pine, box, i, would, sur..."
1284,Drake,God's Plan,[Intro],"Yeah, they wishin' and wishin' and wishin' and...",Drake,"[yeah, they, wishin', and, wishin', and, wishi..."
1285,Drake,God's Plan,[Verse 1],"I been movin' calm, don't start no trouble wit...",Drake,"[i, been, movin', calm, don't, start, no, trou..."
1286,Drake,God's Plan,[Chorus 1],"God's plan, God's plan, I hold back, sometimes...",Drake,"[god's, plan, god's, plan, i, hold, back, some..."
1287,Drake,God's Plan,[Post-Chorus],"And still, bad things, It's a lot of bad thing...",Drake,"[and, still, bad, things, it's, a, lot, of, ba..."


In [8]:
from nltk.tokenize import word_tokenize
model_df['tokenize_verse'] = model_df['verse'].copy()
model_df['tokenize_verse'] = model_df['tokenize_verse'].str.lower()
model_df['tokenize_verse'] = model_df['tokenize_verse'].apply(lambda x: word_tokenize(x))

In [9]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
1282,Drake,Buried Alive (Interlude),[Intro],"If you was in a pine box, I would surely break...",Drake,"[if, you, was, in, a, pine, box, i, would, sur...","[if, you, was, in, a, pine, box, ,, i, would, ..."
1284,Drake,God's Plan,[Intro],"Yeah, they wishin' and wishin' and wishin' and...",Drake,"[yeah, they, wishin', and, wishin', and, wishi...","[yeah, ,, they, wishin, ', and, wishin, ', and..."
1285,Drake,God's Plan,[Verse 1],"I been movin' calm, don't start no trouble wit...",Drake,"[i, been, movin', calm, don't, start, no, trou...","[i, been, movin, ', calm, ,, do, n't, start, n..."
1286,Drake,God's Plan,[Chorus 1],"God's plan, God's plan, I hold back, sometimes...",Drake,"[god's, plan, god's, plan, i, hold, back, some...","[god, 's, plan, ,, god, 's, plan, ,, i, hold, ..."
1287,Drake,God's Plan,[Post-Chorus],"And still, bad things, It's a lot of bad thing...",Drake,"[and, still, bad, things, it's, a, lot, of, ba...","[and, still, ,, bad, things, ,, it, 's, a, lot..."


In [10]:
model_df['verse'] = model_df['verse'].str.lower()
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace("'",''))
model_df['verse'] = model_df['verse'].apply(lambda x: "".join([c if c.isalnum() else " " for c in x ]))
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace('  ',' '))

In [11]:
stopwords = ["yeah", "uh", "huh", "aw", "oh", "la", "ayy",
            "na", "ha", "ay", "skrrt",
            "em", "bla", "woo", "wu", "ya", "wala", "woop", "wrarrr", "ooh",
            "ba", "duh", "du", "1", "2", "3", "4" ,"5" ,"6", "7", "8", "9", "0"]

for index, row in model_df.iterrows():
    querywords = row['verse'].split()

    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    row['verse'] = ' '.join(resultwords)

model_df.verse[:5]

1282    if you was in a pine box i would surely break ...
1284    they wishin and wishin and wishin and wishin t...
1285    i been movin calm dont start no trouble with m...
1286    gods plan gods plan i hold back sometimes i wo...
1287    and still bad things its a lot of bad things t...
Name: verse, dtype: object

In [12]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
1282,Drake,Buried Alive (Interlude),[Intro],if you was in a pine box i would surely break ...,Drake,"[if, you, was, in, a, pine, box, i, would, sur...","[if, you, was, in, a, pine, box, ,, i, would, ..."
1284,Drake,God's Plan,[Intro],they wishin and wishin and wishin and wishin t...,Drake,"[yeah, they, wishin', and, wishin', and, wishi...","[yeah, ,, they, wishin, ', and, wishin, ', and..."
1285,Drake,God's Plan,[Verse 1],i been movin calm dont start no trouble with m...,Drake,"[i, been, movin', calm, don't, start, no, trou...","[i, been, movin, ', calm, ,, do, n't, start, n..."
1286,Drake,God's Plan,[Chorus 1],gods plan gods plan i hold back sometimes i wo...,Drake,"[god's, plan, god's, plan, i, hold, back, some...","[god, 's, plan, ,, god, 's, plan, ,, i, hold, ..."
1287,Drake,God's Plan,[Post-Chorus],and still bad things its a lot of bad things t...,Drake,"[and, still, bad, things, it's, a, lot, of, ba...","[and, still, ,, bad, things, ,, it, 's, a, lot..."


In [13]:
def get_words(df):
    all_words = []
    
    for index, row in df['verse'].iteritems():
        new_words = re.findall(r"\b[a-z']+\b", unidecode(row))
        all_words += new_words
        
    return all_words

In [14]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
1282,Drake,Buried Alive (Interlude),[Intro],if you was in a pine box i would surely break ...,Drake,"[if, you, was, in, a, pine, box, i, would, sur...","[if, you, was, in, a, pine, box, ,, i, would, ..."
1284,Drake,God's Plan,[Intro],they wishin and wishin and wishin and wishin t...,Drake,"[yeah, they, wishin', and, wishin', and, wishi...","[yeah, ,, they, wishin, ', and, wishin, ', and..."
1285,Drake,God's Plan,[Verse 1],i been movin calm dont start no trouble with m...,Drake,"[i, been, movin', calm, don't, start, no, trou...","[i, been, movin, ', calm, ,, do, n't, start, n..."
1286,Drake,God's Plan,[Chorus 1],gods plan gods plan i hold back sometimes i wo...,Drake,"[god's, plan, god's, plan, i, hold, back, some...","[god, 's, plan, ,, god, 's, plan, ,, i, hold, ..."
1287,Drake,God's Plan,[Post-Chorus],and still bad things its a lot of bad things t...,Drake,"[and, still, bad, things, it's, a, lot, of, ba...","[and, still, ,, bad, things, ,, it, 's, a, lot..."


In [15]:
get_words(model_df)

['if',
 'you',
 'was',
 'in',
 'a',
 'pine',
 'box',
 'i',
 'would',
 'surely',
 'break',
 'the',
 'lock',
 'id',
 'jump',
 'right',
 'in',
 'and',
 'fall',
 'asleep',
 'cause',
 'you',
 'are',
 'the',
 'death',
 'of',
 'me',
 'if',
 'you',
 'was',
 'in',
 'a',
 'pine',
 'box',
 'i',
 'would',
 'surely',
 'break',
 'the',
 'lock',
 'id',
 'jump',
 'right',
 'in',
 'and',
 'fall',
 'asleep',
 'cause',
 'you',
 'are',
 'the',
 'death',
 'of',
 'me',
 'they',
 'wishin',
 'and',
 'wishin',
 'and',
 'wishin',
 'and',
 'wishin',
 'they',
 'wishin',
 'on',
 'me',
 'yuh',
 'i',
 'been',
 'movin',
 'calm',
 'dont',
 'start',
 'no',
 'trouble',
 'with',
 'me',
 'tryna',
 'keep',
 'it',
 'peaceful',
 'is',
 'a',
 'struggle',
 'for',
 'me',
 'dont',
 'pull',
 'up',
 'at',
 'am',
 'to',
 'cuddle',
 'with',
 'me',
 'you',
 'know',
 'how',
 'i',
 'like',
 'it',
 'when',
 'you',
 'lovin',
 'on',
 'me',
 'i',
 'dont',
 'wanna',
 'die',
 'for',
 'them',
 'to',
 'miss',
 'me',
 'yes',
 'i',
 'see',
 'the

In [16]:
verses = list(model_df.verse.values)

In [17]:
verses[:10]

['if you was in a pine box i would surely break the lock id jump right in and fall asleep cause you are the death of me if you was in a pine box i would surely break the lock id jump right in and fall asleep cause you are the death of me',
 'they wishin and wishin and wishin and wishin they wishin on me yuh',
 'i been movin calm dont start no trouble with me tryna keep it peaceful is a struggle for me dont pull up at am to cuddle with me you know how i like it when you lovin on me i dont wanna die for them to miss me yes i see the things that they wishin on me hope i got some brothers that outlive me they gon tell the story shit was different with me',
 'gods plan gods plan i hold back sometimes i wont yuh i feel good sometimes i dont dont i finessed down weston road nessed might go down a g o d wait i go hard on southside g yuh wait i make sure that north side eat',
 'and still bad things its a lot of bad things that they wishin and wishin and wishin and wishin they wishin on me bad t

In [18]:
def make_word_dictionary(verses):
    following = {}
    count = {}
    for verse in verses:
        words = verse.split()
        for index, word in enumerate(words[:-1]):
            # ensure the word is in the dictionary
            if word not in following:
                following[word] = []
                count[word] = 0

            # append the next word
            following[word].append(words[index + 1])
            count[word] += 1

        if words:
            last_word = words[-1]
            if last_word not in following:
                following[last_word] = []
                count[last_word] = 0
                
    return count, following

In [19]:
count, following = make_word_dictionary(verses)

In [20]:
def create_bigrams(verses):
    bigrams = {}
    for words in verses:
        words = words.split()
        words = [x for x in words if x]
        for w1, w2, w3 in zip(words[:-2], words[1:-1], words[2:]):
            bigram = (w1,w2)
            if bigram not in bigrams:
                bigrams[bigram] = [w3]
            else:
                bigrams[bigram].append(w3)   
    return bigrams

In [21]:
verses_bigrams = create_bigrams(verses)

In [22]:
def generate_text_from_bigram(following, bigram, num_words, seeds):    
    all_bigrams = list(bigram.keys())
    current_bigram = random.choice(seeds)
    text = [current_bigram]
    for _ in range(num_words - 1):
        if current_bigram in all_bigrams:
            
            next_word = random.choice(bigram[current_bigram])
        else:
            if following[text[-1]]:
                next_word = random.choice(following[text[-1]])
            else:
                continue
        text.append(next_word)
        current_bigram = (current_bigram[1], next_word)
    return ' '.join(text)

In [23]:
seeds = ["bitch", "love", "drugs", "cash", "guns", "club", "friends", "paper", "liquor", "apple",
         "blue", "red", "streets", "people", "brother", "cap"]

generated_raps = []
for _ in range(0,100):
    wordlist = generate_text_from_bigram(following, verses_bigrams, 33, seeds)
    generated_raps.append(wordlist)
    print(wordlist)
    print('\n')

apple how many carrots we can go all night whole crews in here breathe again think its funny you dont wanna hurt nobody did it my flow you should know this is not


cap on i need you to rome and i dare anybody to try and forget about take a step up young f n weezy holla at me and to my girlfriend tell me


club district is poppin like literally every week and promotersll push flyers money exchange hands liars sell you they aint got time on your birthday whos gonna save me when you think about


club and so i put a barrier between both of our lives right now girl what about all your homeboys hang in lenox mall mmm mmm mmm i cannot relate ridin with a


blue they gotta look for but wont find you musta done this before this cant be your knight in shining armor all tires girl they had a talk with my mind made up


club and it was fresh to some considerable degree steppin out the summer man you gotta own the things you cant decide fashion week shit no wonder they tryna put the pen is


brot

guns with the screen cracked still hit me when you throw me high fives they try and cut the tie it severed itself this just the style of music that did it my


club goin out of state ridin with some girls turn their back on the table or basically its fuck everything that weve thrown ive done the station here they get too attached to


brother brought your wifey true but what does it feel i aint speaking in code i got a stylist would agree i m texting but this is why i hear talking i just


apple music thing but we should order in tell g to put a barrier between both of our control its a celebration clap clap bravo lobster and shrimp and a fibula when i


drugs got somethin left to prove i made it im too good to you that that isn t what the math is numbers inflated they all sentimental now you think we got a


bitch we must have never been before but inside im treadin waters steady tryin to record why has it i get money i want to have your eyes closed lost some of them


cap from the ci

In [24]:
columns = ['markov_generated_verses']

drake_markov_df = pd.DataFrame(generated_raps, columns=columns)

In [25]:
drake_markov_df['artist'] = 'Drake'

drake_markov_df

,markov_generated_verses,artist
0,apple how many carrots we can go all night who...,Drake
1,cap on i need you to rome and i dare anybody t...,Drake
2,club district is poppin like literally every w...,Drake
3,club and so i put a barrier between both of ou...,Drake
4,blue they gotta look for but wont find you mus...,Drake
5,club and it was fresh to some considerable deg...,Drake
6,brother drizzy drake maddas jheeze jheeze and ...,Drake
7,paper hip hop hates you and your momma to the ...,Drake
8,cash at least tell you im with you dont unders...,Drake
9,bitch i im ready for i just pray that you thou...,Drake


In [26]:
with open('data/drake_markov_raps.pkl', 'wb') as picklefile:
    pickle.dump(drake_markov_df, picklefile)